# mvPot: Estimating the Spatial Dependence of Extreme Temperature Anomaly over the Red Sea

This notebook illustrates the functionalities of the package `mvPot` by modelling extreme temperature anomalies in the North of the Red Sea. We present a complete case study including data exploration, marginal and dependence modelling and finally model simulations. `mvPot` includes two implementations of computationally efficient procedures for fitting extremal processes.

This notebook should be seen as a short generic example which can easily be adapted to larger scale data sets. For presentation efficiency, heavy calculations have been pre-computed and stored, so by default the variable `run_heavy_computations` is set to `FALSE`. After the workshop, you can change it to `TRUE` to re-run everything on your workstation, and observe the discrepency in computational cost between approaches.

We first load the subset of the Data Challenge data set and set the environment for parallel computing. 

In [ ]:
# Load dataset
load("DATA_TRAINING_SUBSET.RData")

# Load set of utility functions (e.g. plotting maps)
source("utils.R")

# Number of core tu use for computation (not necessary to have more than one to reproduce example here)
nCores <- 1

# Laod package for parallelisation and create cluster instance
if(nCores > 1){
  library(parallel)
  cl <- makeCluster(nCores,type="FORK")
  clusterSetRNGStream(cl)
} else {
  cl <- NULL
}

# Set to true if you want to run heavy computations (tractable on laptop but too long for the workshop)
run_heavy_computations <- FALSE

Then, we use the package `sp` to properly define the system of coordinates (lon/lat) and project them in the UTM coodinates system (km). If you have a google API key, then we also plot a sample from the data.

In [ ]:
# Load package for manipulation of coordinates
library(sp)

# Create coordinates object
coords <- as.data.frame(loc.sub)
names(coords) <-  c("X", "Y")
coordinates(coords) <- c("X", "Y")

# Define system of coordinates
proj4string(coords) <- CRS("+proj=longlat +datum=WGS84")

# Project lon/lat coordinates to UTM (m) system of coordinates
projected_coordinates <-  spTransform(coords, CRS("+proj=utm +zone=37 ellps=WGS84"))

# Rescale to have new coordinates in km
projected_coordinates@coords <-  projected_coordinates@coords / 1000

#Load ggplot2 for plot
library(ggplot2)

# Set to true if you want to produce maps
map_plotting <- FALSE

if(map_plotting == TRUE){
  # Load package for map plotting
  library(ggmap)
  
  # Specify here your onwn google_api_key
  api <- readLines("google_api_key")
  register_google(key = api)
  
  # Downlaod map
  map <- get_map(location = c(mean(loc.sub[,1]),mean(loc.sub[,2])), maptype = "toner-2010", zoom = 7)
  
  #Plot one sample from the dataset
  mapPlot(anom.training.sub[1,], loc.sub, title = "Temperature Anomaly")
}


## Data Exploration

We have a quick look at some important properties of the data, in particular we have a look at local means and $0.95$ quantiles. Other quantities and quantiles could be considered. You can observe:
* the large quantity of missing data;
* the different spatial structure between the mean and the quantile.

In [ ]:
# Data set summary
print("Data set summary:")
summary(as.vector(anom.training.sub))

# Compute local mean and 0.95quantile
quantile_anomaly <- rep(NA, length(anom.training.sub[1,]))
mean_anomaly <- rep(NA, length(anom.training.sub[1,]))

# Set quantile level
quantileLevel <- 0.95
for(i in 1:length(quantile_anomaly)){
  quantile_anomaly[i] <- quantile(anom.training.sub[,i], quantileLevel, na.rm = TRUE)
  mean_anomaly[i] <- mean(anom.training.sub[,i], na.rm = TRUE)
}

print("Local mean:")
summary(mean_anomaly)
print("Local empirical 0.95 quantile:")
summary(quantile_anomaly)

In [ ]:
#Plot results
if(map_plotting == TRUE){
  mapPlot(mean_anomaly, loc.sub, title = "Local Mean")
}

if(map_plotting == TRUE){
  mapPlot(quantile_anomaly, loc.sub, title = "Local 0.95 quantile")
}

Suppose now that the themperature anomalies have been generated by a stochastic process $X$ taking values in the space of continuous functions $C(S)$ on a compact subset $S \subset \mathbb{R}^2$; the subset $S$ represents in this case the North part of the Red sea. To further the exploration, we now study the local tail behaviour of the temperature anomaly by fitting a generalized Pareto ditribution for every location (grid cell) $s \in S$, i.e.,

$$
\text{Pr}\{X(s) - u(s) > x\} \approx \left\{1 + \xi(s) \frac{x}{\sigma(s)}\right\}^{1/\xi(s)}, \quad x \geqslant 0,
$$

where $u(s)$ is the $0.95$ local empirical quantile at the location $s$. The fit is done here by maximum likelihood estimation as implemented in the package `evd`.

In [ ]:
# Parameters for generalized Pareto distributions
scales <- rep(NA, length(anom.training.sub[1,]))
shapes <- rep(NA, length(anom.training.sub[1,]))

#List of fitted objects to check results
list_fit <- list()

# Local GP fit above the 0.95 quantile
for(i in 1:length(scales)){
  #print(paste(i, "/", length(scales)))
  
  # Likelihood based GPD fit
  fitGP <- evd::fpot(
    anom.training.sub[, i],
    threshold = quantile_anomaly[i],
    std.err = FALSE,
    method = "Nelder-Mead",
    control = list(maxit = 10000)
  )
  
  # Store estimation results
  scales[i] <- fitGP$estimate[1]
  shapes[i] <- fitGP$estimate[2]
  list_fit[[i]] <- fitGP
}

#Check results
summary(shapes)
summary(scales)

We can then display visual diagnotics to locally check the quality of the fit and understand the spatial variability of parameters estimates.

In [ ]:
# Location to plot 
posistion_to_plot <- 500

#Plot fit quality
plot(list_fit[[posistion_to_plot]], which = 2)

In [ ]:
# Plot maximum likelhood tail parameter estimates
if(map_plotting == TRUE){
  mapPlot(shapes, loc.sub, title = "Estimated tail indexes (xi)")
}

In [ ]:
# Plot maximum likelhood scale estimates
if(map_plotting == TRUE){
  mapPlot(scales, loc.sub, title = "Estimated scale parameters (sigma)")
}

## Asmyptotic Distribution of Functional Exceedances

Spatial model for extremes were first introduced for block maxima and modelled using max-stable processes. While attractive, these kind of processes are not suited to model the joint tail behaviour of single extreme events (instead of, for instance, yearly maxima). The limiting distribution of functional exceedances have been studied by Dombry et Ribatet (2015) and they introduce the family of r-Pareto processes that we present here.

First, the notion of exceedances for functions is not as straightforward as for univariate random variables and needs to be carefully introduced. Let $\text{r}: C(S) \rightarrow \mathbb{R}_+$ be a homogeneous functional, i.e., there exists $\alpha \in \mathbb{R}$ such that

$$
\text{r}(\lambda x) = \lambda^\alpha \text{x}, \quad x \in C(S), \; \lambda \in \mathbb{R}_+.
$$

We define a functional exceedance as an event $\{\text{r}(X) \geq u\}$ for a threshold $u \in \mathbb{R}_+$. The functional $\text{r}$ is called a risk functional and characterizes the type of extremes under study. Classical examples of risk functionals are
* $\max_{s \in S} X(s)$ for events where at least one location is exceeding a threshold;
* $\int_{s\in S} X(s)$ spatial accumulation.

Without loss of generality we now suppose that $\text{r}$ is risk functional with $\alpha = 1$.
A stochastic process is regulary varying if there exists a sequence $a_n: S \rightarrow \mathbb{R}_+$ of continuous functions such that

$$
n \text{Pr}\left\{\frac{X}{a_n} \in \cdot \right\} \rightarrow \Lambda(\cdot),
$$

where $\Lambda$ is a measure on $C_0(S) = \{x \in C(S) : x(s) \geq 0\} \setminus \{0\}$. Then for any regularly varing stochastic process $X$, we have

$$
\text{Pr}\left.\left\{\frac{X}{u} \in \cdot \; \; \right| \; \text{r}(X) > u \right\} \rightarrow \text{Pr}\{P \in \cdot \; \;\},
$$

where $P$ belongs the the familly of r-Pareto process, defined as 

$$
P = RW, \quad R,
$$

where $R$ is univariate Pareto variable with parameter tail index $\xi > 0$ and $W$ is a stochastic process with sample paths on $\mathcal{S}(S) = \{x \in C(S)_0 : \|x\|_1 = 1\}$. For sample locations $s_1,\dots,s_l$, $l > 1$, the density function of the $\text{r}$-Pareto process is

$$
f(x) = \frac{\lambda(x)}{\Lambda\{x \in \mathbb{R}_+^l : r(x) \geq 1\}}, \quad x \in \mathbb{R}^l,
$$

where 
$$
\Lambda(\cdot) = \int_{(\cdot)} \lambda(x)dx.
$$

The minimal assumption to derive the limiting distribution of $\text{r}$-exceedances is regular variation which imposes that the stochastic process share a single postive tail index $\xi > 0$ for all $s \in S$. This constrain is not satisfied in this study as we have observed spatial variation of tail index which is mostly negative. To work around this limitation we first stadardize the data to have a common local tail behaviour: marginal distributions of $X$ are modified to have tail index $\xi(s) = 1$ for all $s \in S$; the rescaled process is denoted $X^*$.


The practical impact of such an approach is limited by the fact that the risk functional $\text{r}$ must be applied on $X^*$: we lose the possible physical interpretation of the risk. Generalized r-Pareto processes (de Fondeville and Davison, 2019+) generalizes the work of Dombry et Ribatet (2015) to cover all possible regimes of tail decay and to allow the risk to be defined directly on the original data 

## Marginal Standardization

The marginal distributions are standardized to have $\text{GEV}(1,1,1)$ or $\text{GPD}(0,1,1)$ using a marginal transform for which the cumulative distribution at every grid cell $s \in S$ is modelled with

$
\hat{F}(x) = \left\{
\begin{array}{ll}
F_s^n(x), & x \geq u(s) \\
1 - 0.95 \times \left\{1 + \xi(s) \frac{x - u(s)}{\sigma(s)}\right\}^{-1/\xi(s)}, & x > u(s),
\end{array}
\right.
$

where $F_s^n$ denotes the empirical distribution fuction at location $s$.

In [ ]:
# Create object for normalized data
normalized_database <- matrix(NA, nrow = nrow(anom.training.sub), ncol = ncol(anom.training.sub))

# Standarize to unit Frechet or generalized Pareto with unit tail and scale and zero location
frechet = FALSE

for(i in 1:length(scales)){
  # Compute local empirical CDF
  empiricalCdf <- ecdf(anom.training.sub[,i])
  
  if(frechet == TRUE){
    # Use empirical cdf below the threshold
    normalized_database[(anom.training.sub[,i] <= quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i] <- -1 / log(empiricalCdf(anom.training.sub[(anom.training.sub[,i] <= quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i]))
    # Use estimated GP distribution above the threshold
    normalized_database[(anom.training.sub[,i] > quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i] <-
      -1 / log(1 - (1 - quantileLevel) * (1 + shapes[i]*(anom.training.sub[(anom.training.sub[,i] > quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i] - quantile_anomaly[i]) / scales[i])^(-1 / shapes[i]))
  } else {
    # Use empirical cdf below the threshold
  normalized_database[(anom.training.sub[,i] <= quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i] <- 1 / (1 - empiricalCdf(anom.training.sub[(anom.training.sub[,i] <= quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i])) - 1
  # Use estimated GP distribution above the threshold
  normalized_database[(anom.training.sub[,i] > quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i] <- 
    1 /  ((1 - quantileLevel) * (1 + shapes[i]*(anom.training.sub[(anom.training.sub[,i] > quantile_anomaly[i] & !is.na(anom.training.sub[,i])),i] - quantile_anomaly[i]) / scales[i])^(-1 / shapes[i])) - 1
  }
}

#Plot one sample
if(map_plotting == TRUE){
  sample_position <- 1
  mapPlot(normalized_database[sample_position,], loc.sub)
}


## Modelling Extremal Dependence of r-Pareto processes

To model the dependence of the r-Pareto processe, we need to specify the distribution of its angular component $W$. Log-Gaussian random functions are convenient as they allow to use classical dependence models from the literature on spatial statstics to model extremal dependence. In this case the angular part is

$$
W = \frac{\exp G}{\|\exp G\|_1},
$$

where $G$ is a Gaussian process with mean $\text{var}\{G(s)\} / 2$, $s \in S$ and stationary increments, i.e.,

$$
{\text{var}\{G(s') - G(s)\}} = 2\gamma(h), \quad h = s'-s, \quad s,s' \in S,
$$

meaning that the variance of increments depends only on the distance between the sites. The function $\gamma$ is called a semi-variogram and characterizes the dependence of the process.

With this choice of angular process, the intensity function $\lambda$ of the associated r-Pareto process sampled as locations $s_1,\dots,s_l \in S$, $l > 1$ is (Engleke et al., 2015),

$$
\lambda(x_1,\dots x_l) = \frac{\left|\Sigma^{-1/2}\right|}{x_1^2x_2\dots x_l (2\pi)^{(l-1)/2}} \exp \left(-\frac{1}{2} \tilde{x}^T\Sigma^{-1}\tilde{x} \right), \quad x \in \mathbb{R}^l,
$$

where $\tilde{x_i} = \log(x_i/x_1) + \gamma(s_i - s_1)$ and $\Sigma$ is the covariance matrix of the vector $\{G(s_2),\dots, G(s_l)\}$.

## Variogram Estimation with Maximum Censored Likelihood

Wasdworth & Tawn (2014) propose an estimator for r-Pareto process with risk functional $\max_{s \in S}$ in which components below a high theshold $u > 0$ are censored, yielding the intensity function

$$
\lambda_{\text{cens}}(x) = \lambda(x_1,\dots, x_k) \times \Phi_{l -k}\{\mu_{\text{cens}},\Sigma_{\text{cens}}\}, x \in \mathbb{R}^l
$$

where $k > 1$ is the number of uncensored components and $\Phi$ is the distribution of a multivariate normal random vector with zero mean and covariance $\Sigma_{\text{cens}}$, see Engelke et al. (2015) for expressions of $\mu_{\text{cens}}$ and $\Sigma_{\text{cens}}$. Similarly, for this choice of risk functional, the scaling constant $\Lambda\{x \in \mathbb{R}_+^l : r(x) \geq 1\}$ can be written as

$$
\Lambda\{x \in \mathbb{R}_+^l : \max_{s_1,\dots, s_l}(x) \geq 1\} = \sum_{i = 1}^l \Phi_{l -1}(\eta_i,R_i),
$$

see Huser and Davison (2013) for expressions of $\eta_i$ and $R_i$, $i = 1 ,\dots,l$.

Note that computation of a censored likelihood requires multiple evaluation of multivariate distributions function in potentially high dimensions. In de Fondeville & Davison (2018), we discuss and implement a computationally efficient algorithm based on quasi-Monte Carlo estimation to approximate these integrals.

This code is scalable on a cluster to benefit from parallel computing making inference for several hundreds of dimensions tractable. However for a reasonable computation time on a single core, we need to reduce the dimensionality of the dataset. Here we take a random sample of $25$ locations.

In [ ]:
# Define threshold for censoring on rescaled data
if(frechet == TRUE){
    rescaled_threshold <- evd::qgev(0.95,1,1,1)
} else {
    rescaled_threshold <- evd::qgpd(0.95,0,1,1)
 }

#Set seed for subset selection; full censored likelihood is really computationnally heavy and
# a vector of size 1254 is not tractable on a laptop
set.seed("294")

# Define subset size
subset_location_size <- 25
# Random subset selection
index_for_estimation <- sample(size = subset_location_size, x = 1:length(quantile_anomaly), replace = FALSE)

# Subset of locations
locations_for_estimation <- projected_coordinates@coords[index_for_estimation,]

# Subset of data
database_for_estimation <- normalized_database[,index_for_estimation]

# Plot one sample of the database subset
if(map_plotting == TRUE){
  sample_position <- 500
  mapPlot(database_for_estimation[sample_position,],loc.sub[index_for_estimation,],
          lims = c(min(normalized_database[sample_position,], na.rm = TRUE), max(normalized_database[sample_position,], na.rm = TRUE)),
         title = "Location subset")
}


The data set has many missing data, which could be handled by integrating out the corresponding components of the vector. Here, for simplicty we reduce the data base to only keep observations without missing data.

For the semi-variogram function, we choose the parametric model (Schlather and Moreva, 2017)

$$
\gamma(h) = \frac{\left(1 + \|A h \|_2^\alpha\right)^{\beta / \alpha}}{2^{\beta/\alpha} - 1}, \quad \beta \leq 2, 0< \alpha \leq 2,
$$

where $A$ is the anisotropy matrix

$$
A = \left[\begin{array}{cc} cos(\theta) & -\sin(\theta) \\ \kappa \sin(\theta) & \kappa \cos(\theta) \end{array}\right].
$$

For the workshop, the optimization has been pre-computed as it takes about $3$ hours on a single core.


In [ ]:
# Find samples without missing data
observations_without_na <- apply(database_for_estimation,1, function(x){any(is.na(x))})

# Subset database to keep observations without missing data
database_for_estimation <- database_for_estimation[!observations_without_na,]

# Find observations for wich at least one component is exceeding a threshold
database_excesses <- database_for_estimation[apply(database_for_estimation, 1, function(x){any(x > rescaled_threshold)}),]

# Transform matrix of observation to list
list_database_excesses <- lapply(1:length(database_excesses[,1]), function(i){database_excesses[i,]})

# Define variogram model - Here the model by Schlather and Moreva
vario <- function(h, alpha = 1, beta = 1, lambda = 1, A = matrix(c(1,0,0,1),2,2)){
  if(norm(h,type = "2") > 0){
  ((1 + (sqrt(norm(A %*%h,type = "2")^2 / lambda))^alpha )^(beta / alpha) - 1) / (2^(beta / alpha) - 1)
    } else {0}
}

# Create objective function to be minimized
objectiveFunctionCensored = function(parameter, excesses, loc, vario, rescaled_threshold, nCores, cl){
  # Print parameters to follow the evolution of the optimition
  print(parameter)
  
  # Enforce boundaries on the variogram parameters
  if(parameter[1] < 0 | parameter[1] > 2 | parameter[2] > 2 | parameter[3] < 0){return(1e50)}
  if(parameter[4] < -pi/2 | parameter[4] > pi/2 | parameter[5] < 0){return(1e50)}
  
  # Set the variogram parameters
  A <- matrix(c(cos(parameter[4]), -sin(parameter[4]), parameter[5] * sin(parameter[4]),parameter[5] * cos(parameter[4])), 2, 2)
  variogram_model <- function(h){
    vario(h, alpha = parameter[1], beta = parameter[2], lambda = parameter[3], A = A)
  }
  
  # Compute consored likelihood from mvPot package
  mvPot::censoredLikelihoodBR(excesses, loc, variogram_model, rescaled_threshold, p = 4999, vec = mvPot::genVecQMC(4999, length(loc[,1]))$genVec, nCores = nCores, cl = cl)
}

if(run_heavy_computations == TRUE){
# Set seed for quasi Monte Carlo estimate and `ensure' computation time
set.seed(56892734)

# Optimize the objective function to estimate variogram parameters using censored likelihood takes about 3 hours on one core 
ref_time <- proc.time()
estimates_censored_likelihood <- optim(par = c(1.7726726, -4.7176393, 9325.3082619, 1.2818892, 0.7136902),
                     fn = objectiveFunctionCensored,
                     excesses = list_database_excesses,
                     loc = locations_for_estimation,
                     vario = vario,
                     nCores = nCores,
                     cl = cl,
                     rescaled_threshold = rep(rescaled_threshold, subset_location_size),
                     control = list(maxit = 100000, trace = 3),
                     method = "Nelder-Mead")
final_time <- proc.time() - ref_time

# Save results
save(estimates_censored_likelihood, file = "estimated_dependence.RData")
} else {
  load("estimated_dependence.RData")
}

## Model Validation

We now check the relevance of the estimated dependence model. To do so, we use a measure of extremal dependence called the extremogram and defined by

$$
\pi(h) = \lim_{u \rightarrow \infty} \text{Pr}\left\{X(s') > u \left| \; X(s) > u \right\}\right., \quad h = s-s', \quad s,s'\in S.
$$

For r-Pareto processes with log-Gaussian random functions defined earlier, the extremogram is

$$
\pi(h) = 2 \left[ 1 - \Phi\left\{\sqrt{\frac{\gamma(h)}{2}}\right\}\right],
$$

where $\Phi$ is the cumulative distribution function of a standard normal random variable.

To check the quality of the model, we compare the theoretical extremogram, obtained by pluging in the estimated semi-variogram function, to empirical estimates computed with

$$
\hat\pi(h) = \frac{\sum_{j = 1}^n 1{\{X_j(s') > u(s') , X_j(s) > u(s)\}}}{\sum_{j = 1}^n 1{\{X_j(s) > u(s)\}}}.
$$

In [ ]:
# Compute all possible pairwise combinations
pairs_combination_indexes <- expand.grid(1:subset_location_size,1:subset_location_size)

# Create vector of empirical extremogram
empirical_extremogram <- rep(NA, length(pairs_combination_indexes[,1]))

# Compute empirical extremogram for each pair of locations
for(k in 1:length(empirical_extremogram)){
  empirical_extremogram[k] <- computeExtremalCoeff(pairs_combination_indexes[k,1], pairs_combination_indexes[k,2], anom.training.sub[,index_for_estimation], quantile_anomaly[index_for_estimation])
}

# Compute vector of distance between each pairs
distances_between_pairs <- sqrt((projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,1]],1] - projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,2]],1])^2 +
                 (projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,1]],2] - projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,2]],2])^2)

# Compute orientation of each pairs
matrixOfAngles <- atan((projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,1]],2] - projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,2]],2]) /
                         (projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,1]],1] - projected_coordinates@coords[index_for_estimation[pairs_combination_indexes[,2]],1])) / pi * 180
matrixOfAngles[is.nan(matrixOfAngles)] <- 0

# Store results in data frame for easier plotting
estimates <- data.frame(extremogram = empirical_extremogram[!is.nan(empirical_extremogram)], dist = distances_between_pairs[!is.nan(empirical_extremogram)], angle = matrixOfAngles[!is.nan(empirical_extremogram)],
                        x = distances_between_pairs[!is.nan(empirical_extremogram)] * cos(matrixOfAngles[!is.nan(empirical_extremogram)]), y = distances_between_pairs[!is.nan(empirical_extremogram)] * sin(matrixOfAngles[!is.nan(empirical_extremogram)]))

### Empirical extremogram plots

We display the empirical estimates of the extremogram as function of the distance.

In [ ]:
# Plot empirical extremogram as function of distance
p_Extremogram_Dist <- ggplot(data = estimates, aes(x = dist, y = extremogram, color = angle)) + geom_point(alpha = 0.5) +
  guides(color = guide_colorbar(barheight = 10, barwidth = 1)) + scale_color_gradientn(colors = rev(rainbow(7))) + ylim(0,1)
print(p_Extremogram_Dist)

In [ ]:
# Plot empirical extremogram as a map
p_Extremogram_Map <- ggplot(data = estimates, aes(x = x, y = y, color = extremogram)) + geom_point(alpha = 0.5) +
  guides(color = guide_colorbar(barheight = 10, barwidth = 1)) + scale_color_gradientn(colors = rev(rainbow(7)), limits = c(0,1))
print(p_Extremogram_Map)

### Empirical extremogram with estimated model

We display the theoretical value of $\pi$ from the estimated dependence model along with empirical estimates.

In [ ]:
# Define variogram with estimated parameters
variogram_model_censored <- function(h){
  # Anisotrpy matrix
  A <- matrix(c(cos(estimates_censored_likelihood$par[4]), -sin(estimates_censored_likelihood$par[4]), estimates_censored_likelihood$par[5] * sin(estimates_censored_likelihood$par[4]),
                estimates_censored_likelihood$par[5] * cos(estimates_censored_likelihood$par[4])), 2, 2)
  #Variogram function
  vario(h, alpha = estimates_censored_likelihood$par[1], beta = estimates_censored_likelihood$par[2], lambda = estimates_censored_likelihood$par[3], A = A)
}

# Compute theoretical extremogram from variogram
extremogram_model_censored <- rep(0,length(distances_between_pairs))
for(i in 1:length(distances_between_pairs)){
  # Vector of coordinates difference between pairs
  coordinates_difference <- c(distances_between_pairs[i] * cos(matrixOfAngles[i]), distances_between_pairs[i] * sin(matrixOfAngles[i]))
  # Compute extremogram between pairs
  extremogram_model_censored[i] <- 2 * (1 - pnorm(sqrt(variogram_model_censored(coordinates_difference) / 2)))
}

# Plot empirical variogram cloud along with estimated model
pExtremogram <- ggplot(data = estimates, aes(x = dist, y = extremogram, color = angle)) + geom_point(alpha = 0.2) + geom_smooth(se = FALSE) + 
  geom_line(data = data.frame(extremogram = extremogram_model_censored, dist = distances_between_pairs, angle = matrixOfAngles), mapping =  aes(x = dist, y = extremogram, color = angle)) +
  guides(color = guide_colorbar(barheight = 30, barwidth = 1)) + scale_color_gradientn(colors = rev(rainbow(7))) + ylim(0,1)
print(pExtremogram)


## Variogram Estimation with Gradient Scoring Rule 

Censored likelihood is an efficient and robust methodology to estimate the dependence of r-Pareto processes associated to the risk functioan $\text{r} = \sup_{s \in S} X(s)$. However, it requires very intensive computations which limits the dimensionality of the data that can be handled.

In de Fondeville and Davison (2018), we develop an computationally cheap alternative based on score matching (Hyvarinen, 2007): instead on maximizing the log-likelihood function, we minimize

$$
\delta(f) = \sum_{i = 1}^l 2w_i(x)\frac{\partial w_l(x)}{\partial x_l}\frac{\partial \log f(x)}{\partial x_l} + w_l(x)^2\left[\frac{\partial^2 \log f(x)}{\partial x_l^2} + \frac{1}{2}\left\{\frac{\partial \log f(x)}{\partial x_l}\right\}^2\right],
$$

where $w: \mathbb{R}^l_+ \rightarrow \mathbb{R}^l_+ $ is a weighting function vanishing at the boundaries of $\{x \in \mathbb{R}^l_+ : \text{r}(x) \geq 1\}$. By taking the partial deritive of the log density function, the scaling constant of the density function disappears greatly reducing the computational burden. This approach applies to any differentiable risk functional.

### Model estimation

In this case, the optimization takes about few seconds and thus is not pre-computed.

In [ ]:
# Defined weighting function
weighting_function <- function(x, u){
  x * (1 - exp(-(sum(x) / u - 1)))
}

# Partial derivative of weighting function
weighting_function_derivative <- function(x, u){
  (1 - exp(-(sum(x) / u - 1))) + (x / (u)) * exp( - (sum(x) / u - 1))
}

# Define objective function for gradient score
objective_function_gradient_score = function(parameter, excesses, locations_coordinates, vario, weighting_function, weighting_function_derivative, threshold, nCores, cl = NULL){
  
  # Print parameter to follow optimization evolution
  print(parameter)

  #Define the variogram corresponding to input parameters
  if(parameter[1] < 0 | parameter[1] > 2 | parameter[2] > 2 | parameter[3] < 0){return(1e50)}
  if(parameter[4] < -pi/2 | parameter[4] > pi/2 | parameter[5] < 0){return(1e50)}
  A <- matrix(c(cos(parameter[4]), -sin(parameter[4]), parameter[5] * sin(parameter[4]),parameter[5] * cos(parameter[4])), 2, 2)
  variogram_model <- function(h){
    vario(h, alpha = parameter[1], beta = parameter[2], lambda = parameter[3], A = A)
  }
  
  #Compute score
  mvPot::scoreEstimation(excesses, locations_coordinates, variogram_model, weighting_function, weighting_function_derivative, u = threshold, nCores = nCores, cl = cl)
}

# Compute spatial accumulation for each rescaled observations
sums <- apply(database_for_estimation,1, sum)

#Defined threshold for exceedance - We chose 0.9 quantile to obtain about the same number of observations as for censored likelihood
threshold <- quantile(sums, 0.9)

# Create matrix of exceedances for spatial accumulation
spatial_accumulation_excesses <- database_for_estimation[sums > threshold,]
list_spatial_accumulation_excesses <- lapply(1:length(spatial_accumulation_excesses[,1]), function(i){spatial_accumulation_excesses[i,]})

#Estimate the parameter - Quick to run on single core
estimates_gradient_score <- optim(par = c(1.9686318,   -2.5746466, 3917.9238753,    0.5421206,    0.5714497),
                     fn = objective_function_gradient_score,
                     excesses = list_spatial_accumulation_excesses,
                     locations_coordinates = as.data.frame(locations_for_estimation),
                     vario = vario,
                     weighting_function = weighting_function,
                     weighting_function_derivative = weighting_function_derivative,
                     threshold = threshold,
                     nCores = nCores,
                     cl = cl,
                     control = list(maxit = 10000, trace = 1),
                     method = "Nelder-Mead")


save(estimates_gradient_score, file = "gradient_score_estimate.RData")


### Model validation

Similary to the model obatained using the censored likelihood estimator, we display here the theoretical value of the extremogram using the estimated estimated model along with empirical estimates. 

In [ ]:
# Define variogram with estimated parameters
variogram_model_score <- function(h){
  # Anisotrpy matrix
  A <- matrix(c(cos(estimates_gradient_score$par[4]), -sin(estimates_gradient_score$par[4]), estimates_gradient_score$par[5] * sin(estimates_gradient_score$par[4]),
                estimates_gradient_score$par[5] * cos(estimates_gradient_score$par[4])), 2, 2)
  # Variogram function
  vario(h, alpha = estimates_gradient_score$par[1], beta = estimates_gradient_score$par[2], lambda = estimates_gradient_score$par[3], A = A) 
}

# Compute theoretical extremogram from variogram
extremogram_model_score <- rep(0,length(distances_between_pairs))
for(i in 1:length(distances_between_pairs)){
  # Vector of coordinates difference between pairs
  coordinates_difference <- c(distances_between_pairs[i] * cos(matrixOfAngles[i]), distances_between_pairs[i] * sin(matrixOfAngles[i]))
  # Compute extremogram between pairs
  extremogram_model_score[i] <- 2 * (1 - pnorm(sqrt(variogram_model_score(coordinates_difference) / 2)))
}

# Plot empirical variogram cloud along with estimated model
pExtremogram <- ggplot(data = estimates, aes(x = dist, y = extremogram, color = angle)) + geom_point(alpha = 0.2) + geom_smooth(se = FALSE) + 
  geom_line(data = data.frame(extremogram = extremogram_model_score, dist = distances_between_pairs, angle = matrixOfAngles), mapping =  aes(x = dist, y = extremogram, color = angle)) +
  guides(color = guide_colorbar(barheight = 10, barwidth = 1)) + scale_color_gradientn(colors = rev(rainbow(7))) + ylim(0,1)
print(pExtremogram)


## Simulating from Fitted Models

Finally, we use the `mvPot` function `simulPareto` to simulate from both models. We use the exact same seed for both models to make their differences as clear as possible.

The simulation is done on the overall field, and takes few minutes as it requires the inversion of a dense matrix of size $1254 \times 1254$, and thus has been pre-computed for the workshop.

In [ ]:
# Produce simulation with same seed for comparison
if(run_heavy_computations == TRUE){
set.seed(1093)
simulation_censored <- mvPot::simulPareto(n = 1, loc = as.data.frame(projected_coordinates@coords), vario = variogram_model_censored, nCores = 1)
set.seed(1093)
simulation_score <- mvPot::simulPareto(n = 1, loc = as.data.frame(projected_coordinates@coords), vario = variogram_model_score, nCores = 1)
save(simulation_censored, simulation_score, file = "simulations.RData")
} else {
  load("simulations.RData")
}

# Marginal back-transform
if(frechet == TRUE){
  simulation_censored_uniform <- evd::pgev(simulation_censored[[1]], 1,1,1)
  simulation_score_uniform <- evd::pgev(simulation_score[[1]], 1,1,1)
} else{
  simulation_censored_uniform <- evd::pgpd(simulation_censored[[1]], 0,1,1)
  simulation_score_uniform <- evd::pgpd(simulation_score[[1]], 0,1,1)
}

simulation_censored_rescaled <- rep(NA, length(simulation_censored[[1]]))
simulation_score_rescaled <- rep(NA, length(simulation_score[[1]]))

for(i in 1:length(simulation_score_rescaled)){
  if(simulation_censored_uniform[i] < 0.95){simulation_censored_rescaled[i] <- quantile(anom.training.sub[,i], probs =  simulation_censored_uniform[i], na.rm = TRUE)} else {
    simulation_censored_rescaled[i] <- evd::qgpd((simulation_censored_uniform[i] - 0.95) / 0.05, loc = quantile_anomaly[i], scale = scales[i], shape = shapes[i])
  }
  if(simulation_score_uniform[i] < 0.95){simulation_score_rescaled[i] <- quantile(anom.training.sub[,i], probs =  simulation_score_uniform[i], na.rm = TRUE)} else {
    simulation_score_rescaled[i] <- evd::qgpd((simulation_score_uniform[i] - 0.95) / 0.05, loc = quantile_anomaly[i], scale = scales[i], shape = shapes[i])
  }
}

print("Summary censored likelihood")
summary(simulation_censored_rescaled)
print("Summary gradient scoring rule")
summary(simulation_score_rescaled)

In [ ]:
# Plot simulations
plot_limits <- c(min(c(simulation_censored_rescaled, simulation_score_rescaled)), max(c(simulation_censored_rescaled, simulation_score_rescaled)))
if(map_plotting == TRUE){
  mapPlot(simulation_censored_rescaled, loc.sub, title = "Censored likelihood", lims = plot_limits)
}

if(map_plotting == TRUE){
  mapPlot(simulation_score_rescaled, loc.sub, title = "Gradient Scoring Rule", lims = plot_limits)
}


## References

Dombry, C., & Ribatet, M. (2015). Functional Regular Variations, Pareto Processes and Peaks Over Thresholds. Statistics and Its Interface, 8(1), 9–17.

Engelke, S., Malinowski, A., Kabluchko, Z., & Schlather, M. (2015). Estimation of Hüsler--Reiss Distributions and Brown--Resnick Processes. Journal of the Royal Statistical Society. Series B (Statistical Methodology), 77(1), 239–265.

de Fondeville, R., & Davison, A. C. (2019+). Functional Peaks-over-threshold Analysis and Generalized r-Pareto Processes. Preprint.

de Fondeville, R., & Davison, A. C. (2018). High-dimensional Peaks-over-threshold Inference. Biometrika, 105(3), 575–592.

Huser, R., & Davison, A. C. (2013). Composite Likelihood Estimation for the Brown--Resnick Process. Biometrika, 100(2), 511–518.

Schlather, M., & Moreva, O. (2017). A Parametric Model Bridging Between Bounded and Unbounded Variograms. Stat, 6(1), 47–52.

Wadsworth, J. L., & Tawn, J. A. (2014). Efficient Inference for Spatial Extreme Value Processes Associated to Log-Gaussian Random Functions. Biometrika, 101(1), 1–15.
